In [ ]:
import sys
sys.path.append("../../")
import csv
import matplotlib.pyplot as plt
import numpy as np
import focusadd
from focusadd.surface.Surface import Surface
from focusadd.coils.CoilSet import CoilSet
from focusadd.lossFunctions.DefaultLoss import default_loss
import numpy as np
import mayavi as maya
from mayavi import mlab
from functools import partial
mlab.init_notebook('x3d',800,800)

In [ ]:
def draw_surface(surface):
    r = surface.get_r()
    x = r[:,:,0]
    y = r[:,:,1]
    z = r[:,:,2]
    p = mlab.mesh(x,y,z,color=(0.8,0.0,0.0))
    return p

def draw_r_centroid(r_central, color = "blue"):
    if color.lower() == "blue":
        tup = (0.0, 0.0, 0.8)
    elif color.lower() == "red":
        tup = (0.8, 0.0, 0.0)
    else:
        tup = (0.0, 0.8, 0.0)
    for ic in range(r_central.shape[0]):
        p = mlab.plot3d(r_central[ic,:,0], r_central[ic,:,1], r_central[ic,:,2], tube_radius = 0.004, line_width = 0.01, color = tup)
    return p

def draw_coils(r_coils, color = "blue"):
    if color.lower() == "blue":
        tup = (0.0, 0.0, 0.8)
    elif color.lower() == "red":
        tup = (0.8, 0.0, 0.0)
    else:
        tup = (0.0, 0.8, 0.0)
    for ic in range(r_coils.shape[0]):
        for n in range(r_coils.shape[2]):
            for b in range(r_coils.shape[3]):
                p = mlab.plot3d(r_coils[ic,:,n,b,0], r_coils[ic,:,n,b,1], r_coils[ic,:,n,b,2], tube_radius = 0.004, line_width = 0.01, color = tup)
    return p

In [ ]:
surface = Surface("../../focusadd/initFiles/axes/ellipticalAxis4Rotate.txt", 128, 32, 1.0)

In [ ]:
coil_data_fil, params_fil = CoilSet.get_initial_data(surface, input_file="filament.hdf5")
_, _, r_fil, _, l_fil = CoilSet.get_outputs(coil_data_fil, False, params_fil)
r_centroid_fil = CoilSet.get_r_centroid(coil_data_fil, False, params_fil)

In [ ]:
coil_data_fb, params_fb = CoilSet.get_initial_data(surface, input_file="fixed_finite_build.hdf5")
_, _, r_fb, _, l_fb = CoilSet.get_outputs(coil_data_fb, False, params_fb)
r_centroid_fb = CoilSet.get_r_centroid(coil_data_fb, False, params_fb)

In [ ]:
coil_data_rot, params_rot = CoilSet.get_initial_data(surface, input_file="rotate_finite_build.hdf5")
_, _, r_rot, _, l_rot = CoilSet.get_outputs(coil_data_rot, False, params_rot)
r_centroid_rot = CoilSet.get_r_centroid(coil_data_rot, False, params_rot)

# Filamentary Coils

In [ ]:
mlab.clf()
p = draw_surface(surface)
p = draw_coils(r_fil, color="blue")
p

# Finite Build Coils, No Rotation

In [ ]:
mlab.clf()
p = draw_surface(surface)
p = draw_coils(r_fb, color="red")
p

# Finite Build Coils, With Rotation

In [ ]:
mlab.clf()
p = draw_surface(surface)
p = draw_coils(r_rot, color="green")
p

# (i) Loss Value Comparison

In [ ]:
fil_f = "filament.txt"
fb_f = "fixed_finite_build.txt"
rot_f = "rotate_finite_build.txt"

In [ ]:
with open(fil_f, 'r') as f:
    reader = csv.reader(f, delimiter=',')
    fil_loss = next(reader)
loss_fil_arr = [float(l) for l in fil_loss]
loss_fil = loss_fil_arr[-1]

In [ ]:
with open(fb_f, 'r') as f:
    reader = csv.reader(f, delimiter=',')
    fb_loss = next(reader)
loss_fb_arr = [float(l) for l in fb_loss]
loss_fb = loss_fb_arr[-1]

In [ ]:
with open(rot_f, 'r') as f:
    reader = csv.reader(f, delimiter=',')
    rot_loss = next(reader)
loss_rot_arr = [float(l) for l in rot_loss]
loss_rot = loss_rot_arr[-1]

In [ ]:
plt.plot(loss_fil_arr)
plt.plot(loss_fb_arr)
#plt.plot(loss_rot_arr)
plt.yscale('log')
plt.ylim([0.25, 1])
plt.show()

The difference is tiny.

# (ii) How much does the coil centroid differ across plots?

Plotting filamentary versus finite-build, no rotate. The filamentary coils are slightly further from the plasma here.

In [ ]:
mlab.clf()
p = draw_surface(surface)
p = draw_r_centroid(r_centroid_fil, color="blue")
p = draw_r_centroid(r_centroid_fb, color="red")
p

Because theta is defined by the axis here, I believe we can actually subtract the two coil sets and take the maximum to get an answer of what the average magnitude of separation is (and what the maximum separation is). 

In [ ]:
diff = r_centroid_fil - r_centroid_fb
abs = np.absolute(diff)
print(np.mean(abs))
print(np.max(abs))

The stellarator has a characteristic major radius of 1.0 and minor radius of 0.1, so the average displacement is about 0.3mm for a 1m device. For a major radius 5.5m device (W7-X) this is about 1.5mm on average. The maximum displacement is about 1.1mm for a 1m device, so about 6mm for a 5.5m device. The key question though is how much does this change the quadratic flux. I'll look at this in a moment. 

Plotting finite-build, no rotate versus finite-build, w/ rotate

In [ ]:
mlab.clf()
p = draw_surface(surface)
p = draw_r_centroid(r_centroid_fb, color="red")
p = draw_r_centroid(r_centroid_rot, color="green")
p

# (iii) What is rotation profile for square coils?

How much does the rotation profile change across plots? Here I plot fb with no rotation versus rb with rotation

In [ ]:
mlab.clf()
p = draw_surface(surface)
p = draw_coils(r_fb, color="red")
p = draw_coils(r_rot, color="green")
p

In [ ]:
fc_fb, fr_fb = params_fb
fc_rot, fr_rot = params_rot
print(np.amax(fr_rot))
print(np.linalg.norm(fr_rot))

Not very much rotation. I'll test non-square coils in a later notebook.

# (iv) The physics loss versus the weight loss

These comparisons are hard to make. I should eventually divide the weight loss by number of coils, to keep things consistent. I'll also want to divide by the B^2 flux for when the number of coils is different, but for fixed coil comparisons this works fine. For now we'll look at it as is.

In [ ]:
w_L = 0.1
w_B = 1e3
w_args = (w_B, w_L)
surface_data = (surface.get_r_central(), surface.get_nn(), surface.get_sg())
coil_output_func_fil = partial(CoilSet.get_outputs, coil_data_fil, False)
loss_fil = default_loss(surface_data, coil_output_func_fil, w_args, params_fil)
coil_output_func_fb = partial(CoilSet.get_outputs, coil_data_fb, False)
loss_fb = default_loss(surface_data, coil_output_func_fb, w_args, params_fb)
f_b_fil = loss_fil - w_L * l_fil
f_b_fb = loss_fb - w_L * l_fb
print(loss_fil)
print(loss_fb)
print(l_fil)
print(l_fb)
print(f_b_fil)
print(f_b_fb)

So the filamentary coils have a slightly longer length but slightly less quadratic flux. Now, the question to ask is: what is the quadratic flux *if* the filamentary coils were built with finite-build? Let's create new params with the centroid values from fil, and the rotation values of zero. Then let's see what they look like.

In [ ]:
fc, _ = params_fil
_, fr = params_fb
params_new = fc, fr
coil_data_new = coil_data_fb
_, _, r_new, _, l_new = CoilSet.get_outputs(coil_data_new, False, params_new)
r_centroid_new = CoilSet.get_r_centroid(coil_data_new, False, params_new)

In [ ]:
surface_data = (surface.get_r_central(), surface.get_nn(), surface.get_sg())
coil_output_func = partial(CoilSet.get_outputs, coil_data_new, False)
loss_new = default_loss(surface_data, coil_output_func, w_args, params_new)

In [ ]:
print(loss_new)
print(loss_fil)
f_b_new = loss_new - w_L * l_new
print(l_new)
print(l_fb)
print(f_b_new)
print(f_b_fb)

In [ ]:
mlab.clf()
p = draw_surface(surface)
p = draw_coils(r_new, color="blue")
p = draw_coils(r_fb, color="red")
p

I see what's happening. The blue coils (filamentary) are relatively further from the plasma, so the quadratic flux is lower. A couple things need to happen to resolve this: the quadratic flux needs to be divided by B^2. The length needs to be divided by the number of coils. The length also needs to not depend on NC and NB, only the coil centroid. This way it is consistent between FB and FIL coils.

# What happens if I increase the surface resolution? Does this change my loss function values?

Let's start with the default surface and get a loss value.

In [ ]:
surface = Surface("../../focusadd/initFiles/axes/ellipticalAxis4Rotate.txt", 128, 32, 1.0)

In [ ]:
coil_data_fil, params_fil = CoilSet.get_initial_data(surface, input_file="filament.hdf5")
_, _, r_fil, _, l_fil = CoilSet.get_outputs(coil_data_fil, False, params_fil)
r_centroid_fil = CoilSet.get_r_centroid(coil_data_fil, False, params_fil)
w_L = 0.1
w_B = 1e3
w_args = (w_B, w_L)
surface_data = (surface.get_r_central(), surface.get_nn(), surface.get_sg())
coil_output_func_fil = partial(CoilSet.get_outputs, coil_data_fil, False)
loss_fil = default_loss(surface_data, coil_output_func_fil, w_args, params_fil)
print(loss_fil)

Now what happens if I increase the resolution by 2 in each direction?

In [ ]:
surface = Surface("../../focusadd/initFiles/axes/ellipticalAxis4Rotate.txt", 256, 64, 1.0)
coil_data_fil, params_fil = CoilSet.get_initial_data(surface, input_file="filament.hdf5")
_, _, r_fil, _, l_fil = CoilSet.get_outputs(coil_data_fil, False, params_fil)
r_centroid_fil = CoilSet.get_r_centroid(coil_data_fil, False, params_fil)
w_L = 0.1
w_B = 1e3
w_args = (w_B, w_L)
surface_data = (surface.get_r_central(), surface.get_nn(), surface.get_sg())
coil_output_func_fil = partial(CoilSet.get_outputs, coil_data_fil, False)
loss_fil = default_loss(surface_data, coil_output_func_fil, w_args, params_fil)
print(loss_fil)

Yikes! Goes up by a lot. Let's check w/ a resolution of 4 in each direction.

In [ ]:
surface = Surface("../../focusadd/initFiles/axes/ellipticalAxis4Rotate.txt", 512, 128, 1.0)
coil_data_fil, params_fil = CoilSet.get_initial_data(surface, input_file="filament.hdf5")
_, _, r_fil, _, l_fil = CoilSet.get_outputs(coil_data_fil, False, params_fil)
r_centroid_fil = CoilSet.get_r_centroid(coil_data_fil, False, params_fil)
w_L = 0.1
w_B = 1e3
w_args = (w_B, w_L)
surface_data = (surface.get_r_central(), surface.get_nn(), surface.get_sg())
coil_output_func_fil = partial(CoilSet.get_outputs, coil_data_fil, False)
loss_fil = default_loss(surface_data, coil_output_func_fil, w_args, params_fil)
print(loss_fil)

In [ ]:
surface = Surface("../../focusadd/initFiles/axes/ellipticalAxis4Rotate.txt", 1024, 64, 1.0)
coil_data_fil, params_fil = CoilSet.get_initial_data(surface, input_file="filament.hdf5")
_, _, r_fil, _, l_fil = CoilSet.get_outputs(coil_data_fil, False, params_fil)
r_centroid_fil = CoilSet.get_r_centroid(coil_data_fil, False, params_fil)
w_L = 0.1
w_B = 1e3
w_args = (w_B, w_L)
surface_data = (surface.get_r_central(), surface.get_nn(), surface.get_sg())
coil_output_func_fil = partial(CoilSet.get_outputs, coil_data_fil, False)
loss_fil = default_loss(surface_data, coil_output_func_fil, w_args, params_fil)
print(loss_fil)